In [2]:
import sagemaker
#sagemaker.local.LocalSession
#Use local session for traning locally other wise use session
#sess=sagemaker.local.LocalSession()
sess=sagemaker.Session(default_bucket="sagemaker-us-east-1-470086202700")
role= sagemaker.get_execution_role()
account =sess.boto_session.client('sts').get_caller_identity()['Account']
region= sess.boto_session.region_name

In [3]:
#dir(sess)
print(sess.default_bucket())

sagemaker-us-east-1-470086202700


In [4]:
#!docker -d --graph=/home/ec2-user/SageMaker

In [5]:
!cat Dockerfile

FROM nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04

ENV DEBIAN_FRONTEND=noninteractive 

RUN apt-get update && apt-get install -y --no-install-recommends \
		ca-certificates \
		curl \
		netbase \
		wget \
       	build-essential \
        	libopencv-dev\
        	libopenblas-dev \
        	nginx
RUN apt-get install -y git

ENV PYTHON_VERSION=3.6
ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
ENV PATH /opt/conda/bin:$PATH

RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.12-Linux-x86_64.sh -O ~/miniconda.sh && \
    /bin/bash ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    conda install -y scipy pillow gunicorn==19.9.0 && \
    conda install -y gevent flask matplotlib && \
    conda install -y pytorch cudatoolkit=10.2  -c pytorch && \
    conda install -y -c conda-forge opencv
RUN pip install -U git+https://github.com/facebookresearch/fvcore.git
RUN pip install torchvision==0.10.0
	
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${

In [6]:
repo_name="pepp-detectron2"
image_tag="pepp-detectron2"
base_job_name="pepp-detectron2"

%env dockerfile Dockerfile

env: dockerfile=Dockerfile


### Below cells sets up enironment variables build dokcer image and push to ECR If the image is already in ECR skip these steps

In [7]:
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

env: account=470086202700
env: region=us-east-1
env: repo_name=pepp-detectron2
env: image_tag=pepp-detectron2


In [8]:
%%sh

aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

Login Succeeded


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [8]:
!docker build -t $image_tag -f $dockerfile .

Sending build context to Docker daemon  3.101GB
Step 1/19 : FROM nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04
10.2-cudnn7-devel-ubuntu18.04: Pulling from nvidia/cuda

53061382: Pulling fs layer 
9b830ba3: Pulling fs layer 
b50f0070: Pulling fs layer 
874d88f7: Pulling fs layer 
d0a5c788: Pulling fs layer 
57efc7f6: Pulling fs layer 
0f23c366: Pulling fs layer 
10b7c3b1: Pulling fs layer 
9cf89bf0: Pulling fs layer 
85cd61e5: Pull complete 4.4MB/554.4MBBExtracting    327MB/736.6MBDigest: sha256:f8cd4c3aec94a77c0e1c2c2f814e8588447cc24866a6cc88fded7fa0f90c0c37
Status: Downloaded newer image for nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04
 ---> 9d0fb339eeda
Step 2/19 : ENV DEBIAN_FRONTEND=noninteractive
 ---> Running in 95c2cf9b27d6
Removing intermediate container 95c2cf9b27d6
 ---> f6c0896c1dd3
Step 3/19 : RUN apt-get update && apt-get install -y --no-install-recommends 		ca-certificates 		curl 		netbase 		wget        	build-essential         	libopencv-dev        	libopenblas-dev         	ngi

The following NEW packages will be installed:
  adwaita-icon-theme curl dconf-gsettings-backend dconf-service fontconfig
  fontconfig-config fonts-dejavu-core gdal-data glib-networking
  glib-networking-common glib-networking-services gsettings-desktop-schemas
  gtk-update-icon-cache hicolor-icon-theme humanity-icon-theme iproute2
  libaec0 libarmadillo8 libarpack2 libatk-bridge2.0-0 libatk1.0-0
  libatk1.0-data libatspi2.0-0 libavahi-client3 libavahi-common-data
  libavahi-common3 libavcodec-dev libavcodec57 libavformat-dev libavformat57
  libavresample-dev libavresample3 libavutil-dev libavutil55 libbluray2
  libbsd0 libcairo-gobject2 libcairo2 libcharls1 libchromaprint1 libcolord2
  libcroco3 libcrystalhd3 libcups2 libcurl3-gnutls libcurl4 libdap25
  libdapclient6v5 libdatrie1 libdbus-1-3 libdc1394-22 libdc1394-22-dev
  libdconf1 libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2
  libdrm-radeon1 libdrm2 libedit2 libelf1 libepoxy0 libepsilon1 libexif-dev
  libexif12 libexpat

Get:33 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libjson-c3 amd64 0.12.1-1.3ubuntu0.3 [21.7 kB]
Get:34 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libxml2 amd64 2.9.4+dfsg1-6.1ubuntu1.4 [664 kB]
Get:35 http://archive.ubuntu.com/ubuntu bionic/main amd64 mime-support all 3.60ubuntu1 [30.1 kB]
Get:36 http://archive.ubuntu.com/ubuntu bionic/main amd64 netbase all 5.4 [12.7 kB]
Get:37 http://archive.ubuntu.com/ubuntu bionic/main amd64 shared-mime-info amd64 1.9-2 [426 kB]
Get:38 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 tzdata all 2021a-0ubuntu0.18.04 [190 kB]
Get:39 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 xkb-data all 2.23.1-1ubuntu1.18.04.1 [325 kB]
Get:40 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdrm-common all 2.4.101-2~18.04.1 [5560 B]
Get:41 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdrm2 amd64 2.4.101-2~18.04.1 [32.3 kB]
Get:42 http://archive.ubuntu.com/ubuntu bionic/main amd64 

Get:111 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsm1 amd64 1.0.13-4build1 [22.4 kB]
Get:112 http://archive.ubuntu.com/ubuntu bionic/main amd64 libmp3lame0 amd64 3.100-2 [136 kB]
Get:113 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libopenjp2-7 amd64 2.3.0-2build0.18.04.1 [145 kB]
Get:114 http://archive.ubuntu.com/ubuntu bionic/main amd64 libopus0 amd64 1.1.2-1ubuntu1 [159 kB]
Get:115 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libshine3 amd64 3.1.1-1 [22.9 kB]
Get:116 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsnappy1v5 amd64 1.1.7-1 [16.0 kB]
Get:117 http://archive.ubuntu.com/ubuntu bionic/main amd64 libspeex1 amd64 1.2~rc1.2-1ubuntu2 [52.1 kB]
Get:118 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libsoxr0 amd64 0.1.2-3 [65.9 kB]
Get:119 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libswresample2 amd64 7:3.4.8-0ubuntu0.2 [55.2 kB]
Get:120 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtheora0 a

Get:195 http://archive.ubuntu.com/ubuntu bionic/main amd64 libnspr4 amd64 2:4.18-1ubuntu1 [112 kB]
Get:196 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libnss3 amd64 2:3.35-2ubuntu2.12 [1220 kB]
Get:197 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler73 amd64 0.62.0-2ubuntu2.12 [800 kB]
Get:198 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpq5 amd64 10.18-0ubuntu0.18.04.1 [108 kB]
Get:199 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqhull7 amd64 2015.2-4 [152 kB]
Get:200 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialite7 amd64 4.3.0a-5build1 [1224 kB]
Get:201 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libxerces-c3.2 amd64 3.2.0+debian-2 [851 kB]
Get:202 http://archive.ubuntu.com/ubuntu bionic/main amd64 odbcinst amd64 2.3.4-1.1ubuntu3 [12.4 kB]
Get:203 http://archive.ubuntu.com/ubuntu bionic/main amd64 odbcinst1debian2 amd64 2.3.4-1.1ubuntu3 [40.4 kB]
Get:204 http://archive.ubuntu.com/ubuntu 

...

Setting up libxss1:amd64 (1:1.2.2-1) ...
Setting up libgdk-pixbuf2.0-0:amd64 (2.36.11-2) ...
Setting up libcurl4:amd64 (7.58.0-2ubuntu3.14) ...
Setting up libsoup2.4-1:amd64 (2.62.1-1ubuntu0.4) ...
Setting up libgd3:amd64 (2.2.5-4ubuntu0.5) ...
Setting up libvdpau1:amd64 (1.1.1-3ubuntu1) ...
Setting up libsoup-gnome2.4-1:amd64 (2.62.1-1ubuntu0.4) ...
Setting up libdapclient6v5:amd64 (3.19.1-2build1) ...
Setting up gtk-update-icon-cache (3.22.30-1ubuntu4) ...
Setting up libxcursor1:amd64 (1:1.1.15-1) ...
Setting up libpango-1.0-0:amd64 (1.40.14-1ubuntu0.1) ...
Setting up libnetcdf13:amd64 (1:4.6.0-2build1) ...
Setting up libatk-bridge2.0-0:amd64 (2.26.2-1) ...
Setting up librest-0.7-0:amd64 (0.8.0-2) ...
Setting up libopenmpi2:amd64 (2.1.1-8) ...
Setting up libxxf86vm1:amd64 (1:1.1.4-1) ...
Setting up libva-x11-2:amd64 (2.1.0-3) ...
Setting up libxrandr2:amd64 (2:1.5.1-1) ...
Setting up libxerces-c3.2:amd64 (3.2.0+debian-2) ...
Setting up libglx-mesa0:amd64 (20.0.8-0ubuntu1~18.04.1) ...

Preparing to unpack .../7-git_1%3a2.17.1-1ubuntu0.8_amd64.deb ...
Unpacking git (1:2.17.1-1ubuntu0.8) ...
Setting up git-man (1:2.17.1-1ubuntu0.8) ...
Setting up less (487-0.1) ...
Setting up libssl1.0.0:amd64 (1.0.2n-1ubuntu5.7) ...
Setting up liberror-perl (0.17025-1) ...
Setting up libxmuu1:amd64 (2:1.1.2-2) ...
Setting up xauth (1:1.0.10-1) ...
Setting up openssh-client (1:7.6p1-4ubuntu0.5) ...
Setting up git (1:2.17.1-1ubuntu0.8) ...
Processing triggers for libc-bin (2.27-3ubuntu1.4) ...
Processing triggers for mime-support (3.60ubuntu1) ...
Removing intermediate container 2320e88cb7a7
 ---> fabf50ccef2a
Step 5/19 : ENV PYTHON_VERSION=3.6
 ---> Running in e292c563f6fc
Removing intermediate container e292c563f6fc
 ---> e15b1ea105f5
Step 6/19 : ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
 ---> Running in 5c5b7a5c71d0
Removing intermediate container 5c5b7a5c71d0
 ---> a1eeac85a6a3
Step 7/19 : ENV PATH /opt/conda/bin:$PATH
 ---> Running in eec6d40539a4
Removing intermediate container eec6d40539a4

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda


zope.interface-5.4.0 | 321 KB    | ########## | 100% 
kiwisolver-1.3.1     | 86 KB     | ########## | 100% 
zope-1.0             | 4 KB      | ########## | 100% 
c-ares-1.17.1        | 113 KB    | ########## | 100% 
tornado-6.1          | 647 KB    | ########## | 100% 
matplotlib-3.3.4     | 25 KB     | ########## | 100% 
glib-2.69.1          | 1.9 MB    | ########## | 100% 
matplotlib-base-3.3. | 6.7 MB    | ########## | 100% 
libxml2-2.9.10       | 1.3 MB    | ########## | 100% 
mkl-service-2.3.0    | 56 KB     | ########## | 100% 
expat-2.4.1          | 211 KB    | ########## | 100% 
cffi-1.14.6          | 224 KB    | ########## | 100% 
nu

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Solving environment: ...working... done
pytorch-1.9.0        | 706.2 MB  | ########## | 100% 
tqdm-4.62.1          | 80 KB     | ########## | 100% 
ninja-1.10.2         | 1.9 MB    | ########## | 100% 
conda-4.10.3         | 3.1 MB    | ########## | 100% 
cudatoolkit-10.2.89  | 540.0 MB  | ########## | 100% 
conda-package-handli | 962 KB    | ########## | 100% 
## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - cudatoolkit=10.2
    - pytorch


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pytorch-1.9.0              |py3.7_cuda10.2_cudnn7.6.5_0       706.2 MB  pytorch
    tqdm-4.62.1                |     pyhd3eb1b0_1          80 KB
    ninja-1.10.2               |       hff7bd54_1         1.9 MB
    conda-4.10.3              

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - opencv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       h516909a_3         398 KB  conda-forge
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    cairo-1.16.0               |       hf32fb01_1         1.0 MB
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    conda-4.10.3               |   py37h89c1867_0         3.1 MB  conda-forge
    ffmpeg-4.0                 |       hcdf2ecd_0        53.3 MB
    freeglut-3.2.1             |       h58526e2_0         164 KB  conda-forge
    graphite2-1.3.13           |    h58526e2_1001         102 KB  conda-

In [20]:
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

In [9]:
 !docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


In [22]:
 !docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

The push refers to repository [470086202700.dkr.ecr.us-east-1.amazonaws.com/pepp-detectron2]

2798572e: Preparing 
ad13ce64: Preparing 
b0fe1c7c: Preparing 
54e8c99f: Preparing 
05ace858: Preparing 
fe96f7fb: Preparing 
a3721a6d: Preparing 
c946eca3: Preparing 
1de96336: Preparing 
e548ab46: Preparing 
b85f72a2: Preparing 
2a8999d1: Preparing 
bf7d3f67: Preparing 
73aeab99: Preparing 
bfc76a5e: Preparing 
35193021: Preparing 
baa7f084: Preparing 
8e8e53d8: Preparing 
2ee1e25b: Preparing 
9b09744f: Layer already exists latest: digest: sha256:13a45e33388139527bae641918dcb43666bd66d6c7630724bb5da55dc1f9c6c0 size: 4536


### Upload data to S3 bucket from local repo if data is already in s3 just provide the paths of s3 

In [23]:
#local_directory = 'local_test/test_dir/input/data'
#output_dir="pepper-segmentation-s-dataset"
#train_input_path      = sess.upload_data(local_directory+'/training/',      key_prefix=output_dir+'/training')
#validation_input_path = sess.upload_data(local_directory+'/validation/', key_prefix=output_dir+'/validation')

In [10]:
train_input_path="s3://sagemaker-us-east-1-470086202700/pepper-segmentation-s-dataset/training"
validation_input_path="s3://sagemaker-us-east-1-470086202700/pepper-segmentation-s-dataset/validation"

### Run Below cell if the image is already built and you know the name otherwise uncomment the commented part

In [11]:
#output_path = 's3://{}/{}/output'.format(sess.default_bucket(), repo_name)
#image_name  = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, repo_name)
output_path="s3://sagemaker-us-east-1-470086202700/pepper-segmentation-s-dataset/output"
image_name="470086202700.dkr.ecr.us-east-1.amazonaws.com/pepp-detectron2"
print(output_path)
print(image_name)

s3://sagemaker-us-east-1-470086202700/pepper-segmentation-s-dataset/output
470086202700.dkr.ecr.us-east-1.amazonaws.com/pepp-detectron2


In [12]:
train_instance_type ="ml.p3.8xlarge"
#train_instance_type="local_gpu"
#train_instance_type='local'

## Set up estimator and run the traning

In [13]:
estimator = sagemaker.estimator.Estimator(
                       image_uri=image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       instance_count=1, 
                       instance_type=train_instance_type,
                       output_path=output_path,
                       sagemaker_session=sess,
                        #input_mode="Pipe",
                       hyperparameters={
                              'epochs': 20,
                              'batch-size': 130,
                              'lr': 0.01}
                        )

In [14]:
estimator.fit({'training': train_input_path, 'validation': validation_input_path})

2021-09-28 16:05:56 Starting - Starting the training job...ProfilerReport-1632845155: InProgress
...
2021-09-28 16:06:47 Starting - Launching requested ML instances............
2021-09-28 16:08:49 Starting - Preparing the instances for training......
2021-09-28 16:09:53 Downloading - Downloading input data...
2021-09-28 16:10:28 Training - Downloading the training image....................................
2021-09-28 16:16:31 Training - Training image download completed. Training in progress.. Succesffuly Imported required Packages!!
CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 8
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ()
  TRAIN: ()
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FOR

#033[09/28 16:16:45 d2.engine.train_loop]: #033Starting training from iteration 0
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
#033[09/28 16:17:01 d2.utils.events]: #033 eta: 0:02:04  iter: 19  total_loss: 0.09467  loss_cls: 0.01339  loss_box_reg: 0.04441  loss_mask: 0.02913  loss_rpn_cls: 0.001137  loss_rpn_loc: 0.007453  time: 0.6956  data_time: 0.0523  lr: 1.9981e-05  max_mem: 8624M
#033[09/28 16:17:14 d2.utils.events]: #033 eta: 0:01:50  iter: 39  total_loss: 0.0892  loss_cls: 0.01365  loss_box_reg: 0.04101  loss_mask: 0.02889  loss_rpn_cls: 0.001308  loss_rpn_loc: 0.005256  time: 0.6908  da

### Deploy the trained model weights

In [29]:
predictor=estimator.deploy(initial_instance_count=1,instance_type="ml.p3.8xlarge")

-------------------------!

In [ ]:
#!pip install opencv-python

In [71]:
from PIL import Image
import cv2
import numpy as np
import json

In [73]:
image_path="local_test/test_dir/input/data/training/21XZ2418_210715_124212_719242_001.jpg"
#img=np.array(Image.open(image_path))
img=cv2.imread(image_path)
img=cv2.resize(img,(img.shape[0]//4,img.shape[1]//4))
if img is None:
    raise("Image cannot be loaded")
img.shape

(512, 384, 3)

In [76]:
data={"images":img.tolist(),"batch_size":1}
json_data=json.dumps(data)
#json_data

In [77]:
res=predictor.predict(json_data)

In [ ]:
## Delete the endpoint if not needed anymore

In [ ]:
predictor.delete_endpoint()